<a href="https://colab.research.google.com/github/Datasci266-Final-Project/Job-Listings/blob/main/Model%20Training%20Notebooks/Model_3b_Trainer_job_bert_(Epochs_4_5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Installs

!pip install pydot --quiet
!pip install transformers==4.37.2 --quiet
!pip install -U imbalanced-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
#@title Imports
import pandas as pd

import numpy as np
import random
import torch

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, TFBertModel

from transformers import logging
logging.set_verbosity_error()

from collections import Counter

from tensorflow.keras.models import load_model
import pickle
import joblib

In [ ]:
# Set seed to 10

seed_value = 10

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/DataSci 266 Project

column_names = ['index','title','text','level','salary']


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1V3QooLePiHR_DaZhbXQhsjmP1Ez5fv5F/DataSci 266 Project


In [ ]:
pd.set_option('display.max_colwidth', 10000)

In [ ]:
# Dictionaries
level_key = {
    "student_intern": 0,
    "entry": 1,
    "junior": 2,
    "mid": 3,
    "senior": 4,
    "executive": 5,
    np.nan: -1
}

# Define salary_key dictionary
# salary_key = {
#     "<45k": 0,
#     "45-65k": 1,
#     "65-85k": 2,
#     "85-110k": 3,
#     "110-150k": 4,
#     "150-200k": 5,
#     ">200k": 6,
#     np.nan: -1
# }

salary_key = {
    "<50k": 0,
    "50-100k": 1,
    "100-150k": 2,
    "150-200k": 3,
    ">200k": 4,
    np.nan: -1
}


In [ ]:
#@title Import Tokens, Labels, Data

# data
with open(r'Model Objects/train_final_df.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open(r'Model Objects/test_data.pickle', 'rb') as handle:
    test_data = pickle.load(handle)
with open(r'Model Objects/val_data.pickle', 'rb') as handle:
    val_data = pickle.load(handle)


# labels
with open(r'Model Objects/Labels/model_2_labels_level_train.pickle', 'rb') as handle:
    train_level_labels = pickle.load(handle)

with open(r'Model Objects/Labels/model_2_labels_salary_train.pickle', 'rb') as handle:
    train_salary_labels = pickle.load(handle)


with open(r'Model Objects/Labels/model_2_labels_level_test.pickle', 'rb') as handle:
    test_level_labels = pickle.load(handle)

with open(r'Model Objects/Labels/model_2_labels_salary_test.pickle', 'rb') as handle:
    test_salary_labels = pickle.load(handle)


with open(r'Model Objects/Labels/model_2_labels_level_val.pickle', 'rb') as handle:
    val_level_labels = pickle.load(handle)

with open(r'Model Objects/Labels/model_2_labels_salary_val.pickle', 'rb') as handle:
    val_salary_labels = pickle.load(handle)


# tokenized inputs
with open(r'Model Objects/Tokenized Inputs: Bert Base/model_2_train_inputs_bert.pickle', 'rb') as handle:
    train_inputs = pickle.load(handle)

with open(r'Model Objects/Tokenized Inputs: Bert Base/test_inputs_bert.pickle', 'rb') as handle:
    test_inputs = pickle.load(handle)

with open(r'Model Objects/Tokenized Inputs: Bert Base/val_inputs_bert.pickle', 'rb') as handle:
    val_inputs = pickle.load(handle)

In [ ]:
#@title BERT

checkpoint = 'jjzha/jobbert-base-cased'
bert_model = TFBertModel.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/603 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
MAX_SEQUENCE_LENGTH = 512

In [ ]:
#@title Define Model

# Masked Loss Function
def masked_loss_function(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, -1))  # Use -1 to represent missing values
    mask = tf.cast(mask, tf.float32)

    y_true = tf.cast(y_true, tf.float32)  # Cast y_true to float32 to match the type of y_pred
    y_pred = tf.cast(y_pred, tf.float32)

    return tf.keras.losses.sparse_categorical_crossentropy(y_true * mask, y_pred * mask)




# BERT Model
def create_bert_multi_output_model(bert_base_model,
                                   trainable=True,
                                 max_sequence_length=MAX_SEQUENCE_LENGTH,
                                   num_level_classes=7,
                                   num_salary_classes=6,
                                 base_hidden_size=200,
                                  level_hidden_size=200,
                                  salary_hidden_size=200,
                                 dropout=0.3,
                                 learning_rate=0.00005,
                                   LEVEL_WEIGHT=0.5,
                                   SALARY_WEIGHT=0.5):

    bert_base_model.trainable = trainable

    # Input layers
    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {
        'input_ids': input_ids,
        'token_type_ids': token_type_ids,
        'attention_mask': attention_mask
    }

    bert_out = bert_base_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]  # Extract CLS token
    pooler_token = bert_out[1]

    base_hidden = tf.keras.layers.Dense(base_hidden_size, activation='relu', name='base_hidden_layer_1')(cls_token)
    base_hidden = tf.keras.layers.Dense(base_hidden_size, activation='relu', name='base_hidden_layer_2')(base_hidden)
    base_out = tf.keras.layers.Dropout(dropout, name='base_dropout_1')(base_hidden)

    # job experience level
    level_hidden = tf.keras.layers.Dense(level_hidden_size, activation='relu', name='level_hidden_layer_1')(base_hidden)
    level_hidden = tf.keras.layers.Dense(level_hidden_size, activation='relu', name='level_hidden_layer_2')(level_hidden)
    level_hidden = tf.keras.layers.Dropout(dropout, name='level_dropout_1')(level_hidden)
    level_classification = tf.keras.layers.Dense(num_level_classes, activation='softmax', name='level_classification')(level_hidden)

    # salary bucket
    salary_hidden = tf.keras.layers.Dense(salary_hidden_size, activation='relu', name='salary_hidden_layer_1')(base_hidden)
    salary_hidden = tf.keras.layers.Dense(salary_hidden_size, activation='relu', name='salary_hidden_layer_2')(salary_hidden)
    salary_hidden = tf.keras.layers.Dropout(dropout, name='salary_dropout_1')(salary_hidden)
    salary_classification = tf.keras.layers.Dense(num_salary_classes, activation='softmax', name='salary_classification')(salary_hidden)



    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask],
                                          outputs=[level_classification, salary_classification])


    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss={'level_classification': masked_loss_function,
                                       'salary_classification': masked_loss_function},
                                 loss_weights=[LEVEL_WEIGHT,
                                               SALARY_WEIGHT],
                                 metrics={'level_classification': 'accuracy',
                                          'salary_classification': 'accuracy'})

    return classification_model




In [ ]:
#@title Create Model and Train - Take 2


# Fresh instance of BERT model
bert_model = TFBertModel.from_pretrained('jjzha/jobbert-base-cased')
multi_output_model = create_bert_multi_output_model(bert_model)

multi_output_model.load_weights('./Model Objects/model_3b_jobbert/model_3b_jobbert_weights')

# Fit the model
multi_output_model_history = multi_output_model.fit(
    train_inputs,
    {"level_classification": train_level_labels, "salary_classification": train_salary_labels},
    validation_data=(val_inputs, {"level_classification": val_level_labels,"salary_classification": val_salary_labels}),
    batch_size=32,
    epochs=2,
    verbose=1,
    shuffle=True
)



Epoch 1/2


1851/1851 [==============================] - 1910s 882ms/step - loss: 1.0518 - level_classification_loss: 0.5551 - salary_classification_loss: 1.5485 - level_classification_accuracy: 0.7609 - salary_classification_accuracy: 0.1481 - val_loss: 1.3864 - val_level_classification_loss: 1.1403 - val_salary_classification_loss: 1.6325 - val_level_classification_accuracy: 0.5902 - val_salary_classification_accuracy: 0.1186
Epoch 2/2
1851/1851 [==============================] - 1628s 880ms/step - loss: 1.0106 - level_classification_loss: 0.4925 - salary_classification_loss: 1.5287 - level_classification_accuracy: 0.7844 - salary_classification_accuracy: 0.1561 - val_loss: 1.3730 - val_level_classification_loss: 1.1017 - val_salary_classification_loss: 1.6444 - val_level_classification_accuracy: 0.6083 - val_salary_classification_accuracy: 0.1183


In [ ]:
#@title Export model weights


# export model weights
multi_output_model.save_weights('./model_3b_jobbert_5_epochs/model_3b_jobbert_weights')




